In [ ]:
# ==============================================================================
# PART 1: COMPLETE GOOGLE COLAB TRAINING SCRIPT
# ==============================================================================
# Paste this entire block into a single cell in a Google Colab notebook
# with a T4 GPU runtime.

# --- 1. SETUP AND INSTALLATIONS ---
print(">>> [1/8] Installing required libraries...")
!pip install kaggle kagglehub tqdm scikit-learn -q

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import zipfile
import random
import shutil
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive, files
import kagglehub

# --- 2. MOUNT GOOGLE DRIVE & CONFIGURE KAGGLE ---
print("\n>>> [2/8] Setting up Google Drive and Kaggle API...")
# Mount Google Drive to save the model
drive.mount('/content/drive')

# Create a project folder in your Drive
DRIVE_PROJECT_DIR = "/content/drive/MyDrive/BloodCellProject_Colab"
DRIVE_MODELS_DIR = os.path.join(DRIVE_PROJECT_DIR, "models")
os.makedirs(DRIVE_MODELS_DIR, exist_ok=True)
print(f"Models will be saved to: {DRIVE_MODELS_DIR}")

# Upload your kaggle.json file
print("\nIMPORTANT: Please upload your 'kaggle.json' file for API access.")
if not os.path.exists('/root/.kaggle/kaggle.json'):
    files.upload()
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Kaggle API configured.")
else:
    print("Kaggle API already configured.")


# --- 3. CONFIGURATION PARAMETERS ---
class Config:
    # --- Paths within the Colab Environment ---
    # The user can modify this path if needed
    KAGGLE_DATASET_PATH_RAW = '/content/data_raw'
    DATA_PATH_SPLIT = '/content/data_split'

    # --- Training Hyperparameters ---
    MODEL_NAME = 'resnet18'
    BATCH_SIZE = 64
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 25
    IMAGE_SIZE = 224
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    # --- Model Save Path in Google Drive ---
    MODEL_SAVE_PATH = os.path.join(DRIVE_MODELS_DIR, f"{MODEL_NAME}_blood_cell_best.pth")

    # Class labels will be determined automatically from the data folders
    NUM_CLASSES = None
    CLASS_LABELS = None

config = Config()

# --- 4. DOWNLOAD & PREPARE DATASET ---
print("\n>>> [3/8] Downloading and unzipping dataset...")
# Download from KaggleHub
download_path = kagglehub.dataset_download("sumithsingh/blood-cell-images-for-cancer-detection")

# Unzip the data
with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(config.KAGGLE_DATASET_PATH_RAW)
print("Dataset extracted.")

# IMPORTANT: Kaggle datasets can have nested folders. We find the correct path.
# This is a common step that needs adjustment depending on the dataset structure.
actual_data_path = os.path.join(config.KAGGLE_DATASET_PATH_RAW, 'blood-cell-images-for-cancer-detection', 'blood-cell-images-for-cancer-detection')
print(f"Actual data path found at: {actual_data_path}")

def split_data(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.15):
    """Splits data into train, val, and test sets."""
    print("\n>>> [4/8] Splitting data into train, validation, and test sets...")
    shutil.rmtree(dest_dir, ignore_errors=True) # Clean up previous runs

    class_names = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dest_dir, split), exist_ok=True)

    for class_name in class_names:
        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(dest_dir, split, class_name), exist_ok=True)

        source_class_path = os.path.join(source_dir, class_name)
        images = os.listdir(source_class_path)
        random.shuffle(images)

        train_end = int(len(images) * train_ratio)
        val_end = train_end + int(len(images) * val_ratio)

        for i, img in enumerate(tqdm(images, desc=f"Splitting {class_name}")):
            if i < train_end:
                split_name = 'train'
            elif i < val_end:
                split_name = 'val'
            else:
                split_name = 'test'
            shutil.copy(os.path.join(source_class_path, img), os.path.join(dest_dir, split_name, class_name))
    print("Data splitting complete.")

split_data(actual_data_path, config.DATA_PATH_SPLIT)

# --- 5. DATALOADERS & AUGMENTATION ---
def get_dataloaders(config):
    """Creates DataLoaders with augmentation."""
    print("\n>>> [5/8] Creating DataLoaders with augmentation...")
    train_transforms = transforms.Compose([
        transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    val_test_transforms = transforms.Compose([
        transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    train_dataset = datasets.ImageFolder(os.path.join(config.DATA_PATH_SPLIT, 'train'), train_transforms)
    val_dataset = datasets.ImageFolder(os.path.join(config.DATA_PATH_SPLIT, 'val'), val_test_transforms)
    test_dataset = datasets.ImageFolder(os.path.join(config.DATA_PATH_SPLIT, 'test'), val_test_transforms)

    # Dynamically set class info
    config.CLASS_LABELS = train_dataset.classes
    config.NUM_CLASSES = len(train_dataset.classes)

    print("\n" + "="*50)
    print(f"IMPORTANT: Classes found: {config.CLASS_LABELS}")
    print("You MUST use this exact list in your Flask app's config file.")
    print("="*50 + "\n")

    train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

# --- 6. MODEL, TRAINING & EVALUATION FUNCTIONS ---
def get_model(config):
    """Initializes the ResNet18 model."""
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, config.NUM_CLASSES)
    return model.to(config.DEVICE)

def evaluate(model, loader, criterion, device):
    """Evaluates the model on a given dataset."""
    model.eval()
    total_loss, all_preds, all_labels = 0.0, [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return total_loss / len(loader.dataset), accuracy_score(all_labels, all_preds)

def train(config):
    """Main training loop."""
    print("\n>>> [6/8] Starting model training...")
    train_loader, val_loader, test_loader = get_dataloaders(config)
    model = get_model(config)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=config.LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, verbose=True)

    best_val_acc = 0.0
    for epoch in range(config.NUM_EPOCHS):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS}"):
            inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        val_loss, val_acc = evaluate(model, val_loader, criterion, config.DEVICE)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), config.MODEL_SAVE_PATH)
            print(f"  ✨ New best model saved to {config.MODEL_SAVE_PATH} with accuracy: {best_val_acc:.4f}")

    print("\n>>> [7/8] Training complete.")
    return test_loader

def final_evaluation(config, test_loader):
    """Performs a final evaluation on the test set with the best model."""
    print("\n>>> [8/8] Performing final evaluation on the test set...")
    model = get_model(config)
    model.load_state_dict(torch.load(config.MODEL_SAVE_PATH))
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(config.DEVICE), labels.to(config.DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=config.CLASS_LABELS, digits=4))


# --- 7. EXECUTION ---
if __name__ == '__main__':
    print(f"Using device: {config.DEVICE}")
    final_test_loader = train(config)
    final_evaluation(config, final_test_loader)
    print("\n✅ All steps completed successfully!")